In [1]:
import os
os.getcwd()

'/Users/Tomo/Desktop/PTAL'

In [2]:
import partridge as ptg
import pandas as pd
from datetime import time
path = 'data/gtfs.zip'

In [33]:
import zipfile
import pandas as pd

with zipfile.ZipFile('data/gtfs.zip') as z:
    calendar = pd.read_csv(z.open('calendar.txt'))
    if 'calendar_dates.txt' in z.namelist():
        calendar_dates = pd.read_csv(z.open('calendar_dates.txt'))
    else:
        calendar_dates = pd.DataFrame()

print(calendar.head())
print(calendar_dates.head())

  service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0     01-100       0        0          0         0       0         0       1   
1     01-160       0        0          0         0       0         1       0   
2     01-170       1        1          1         1       1         0       0   
3     06-100       0        0          0         0       0         0       1   
4     06-160       0        0          0         0       0         1       0   

   start_date  end_date  
0    20250501  20280430  
1    20250501  20280430  
2    20250501  20280430  
3    20250501  20280430  
4    20250501  20280430  
  service_id      date  exception_type
0     09-172  20250502               1
1     09-170  20250502               2
2     01-105  20250503               1
3     01-160  20250503               2
4     06-100  20250503               1


In [34]:
from datetime import datetime

target_date = '2025-05-08'  # 任意の平日

# 年月日をintで表現（例：20240603）
target_date_int = int(target_date.replace('-', ''))

# その日が calendar.txt のどのサービスに該当するかチェック
weekday = datetime.strptime(target_date, '%Y-%m-%d').strftime('%A').lower()

valid_services = calendar[
    (calendar['start_date'] <= target_date_int) &
    (calendar['end_date'] >= target_date_int) &
    (calendar[weekday] == 1)
]['service_id'].unique()

print(f"{target_date} に運行するサービスID:", valid_services)

2025-05-08 に運行するサービスID: ['01-170' '06-170' '09-170' '13-170' '21-170' '25-170' '29-170' '33-170'
 '37-170' '41-170' '45-170' '49-170' '53-170' '57-170' '61-170' '65-170'
 '69-170' '77-170' '81-170' '85-170']


In [35]:
path = 'data/gtfs.zip'
view = {'trips.txt': {'service_id': valid_services.tolist()}}
feed = ptg.load_feed(path, view)

In [3]:
# 1. 最もサービスが多い日を取得
date, service_ids = ptg.read_busiest_date(path)
print(f"Busiest date: {date}")

Busiest date: 2025-05-23


In [4]:
# 2. 該当日のフィードを読み込む
view = {'trips.txt': {'service_id': service_ids}}
feed = ptg.load_feed(path, view)

In [60]:
# 3. 該当サービスの trip_id を取得
valid_trips = feed.trips['trip_id'].unique()

In [61]:
# 4. 該当 trip に属する stop_times を抽出
stop_times = feed.stop_times[feed.stop_times['trip_id'].isin(valid_trips)].copy()

In [62]:
# 時刻の形式を変換（HH:MM:SS → 時刻オブジェクト）
stop_times['departure_time_obj'] = pd.to_timedelta(stop_times['departure_time'])

In [63]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,departure_time_obj


In [52]:
def hms_to_seconds(hms):
    try:
        h, m, s = map(int, str(hms).split(":"))
        return h * 3600 + m * 60 + s
    except:
        return None  # 不正値・NaNへの対処

In [53]:
stop_times['dep_seconds'] = stop_times['departure_time'].apply(hms_to_seconds)

In [54]:
# 8:15 = 29700秒、9:15 = 33300秒
in_window = stop_times[
    (stop_times['dep_seconds'] >= 29700) &
    (stop_times['dep_seconds'] < 33300)
]

In [40]:
# 5. 8:15〜9:15 の時間帯でフィルタ
start_sec = 8 * 3600 + 15 * 60   # 8:15 = 29700秒
end_sec = 9 * 3600 + 15 * 60     # 9:15 = 33300秒
in_window = stop_times[
    (stop_times['departure_time_obj'].dt.total_seconds() >= start_sec) &
    (stop_times['departure_time_obj'].dt.total_seconds() < end_sec)
]

In [58]:
target_date = '2024-06-03'  # 任意の平日（例：月曜）
target_date_int = int(target_date.replace('-', ''))

# calendar.txt を使って、平日に該当する service_id を取得
weekday = datetime.strptime(target_date, '%Y-%m-%d').strftime('%A').lower()

valid_services = calendar[
    (calendar['start_date'] <= target_date_int) &
    (calendar['end_date'] >= target_date_int) &
    (calendar[weekday] == 1)
]['service_id'].unique()

# 再読み込み
view = {'trips.txt': {'service_id': valid_services.tolist()}}
feed = ptg.load_feed(path, view)

In [59]:
# "HH:MM:SS" → 秒数に変換する関数
def hms_to_seconds(hms):
    try:
        h, m, s = map(int, hms.split(":"))
        return h * 3600 + m * 60 + s
    except:
        return None  # 欠損や不正値がある場合

# 変換適用
stop_times['sec'] = stop_times['departure_time'].apply(hms_to_seconds)

# 統計確認
print(stop_times['sec'].describe())

# 8:15〜9:15 に該当するデータ件数を確認
print(stop_times[(stop_times['sec'] >= 29700) & (stop_times['sec'] < 33300)].shape[0])

count       0
unique      0
top       NaN
freq      NaN
Name: sec, dtype: object
0


In [30]:
in_window = stop_times[
    (stop_times['sec'] >= 29700) &
    (stop_times['sec'] <= 33300)   # ← ここを < から <= に変更
]

In [42]:
# 例えば 8:00〜10:00
start_sec = 8 * 3600
end_sec = 10 * 3600

filtered = stop_times[(stop_times['sec'] >= start_sec) & (stop_times['sec'] <= end_sec)]
print(filtered[['departure_time', 'stop_id']].head(10))
print("件数:", filtered.shape[0])

Empty DataFrame
Columns: [departure_time, stop_id]
Index: []
件数: 0


In [43]:
# 変換後の時間の分布を見る
stop_times['sec'] = stop_times['departure_time_obj'].dt.total_seconds()
print(stop_times['sec'].describe())

# 8:15〜9:15に当たる件数を見てみる
print(stop_times[(stop_times['sec'] >= 29700) & (stop_times['sec'] < 33300)].shape[0])

count    301282.000000
mean          0.000049
std           0.000016
min           0.000018
25%           0.000034
50%           0.000049
75%           0.000063
max           0.000090
Name: sec, dtype: float64
0


In [20]:
# 変換後の時間の分布を見る
stop_times['sec'] = stop_times['departure_time_obj'].dt.total_seconds()
print(stop_times['sec'].describe())

count    301334.000000
mean          0.000049
std           0.000016
min           0.000018
25%           0.000034
50%           0.000049
75%           0.000063
max           0.000090
Name: sec, dtype: float64


In [44]:
# 6. 各 stop_id ごとの停車本数をカウント
stop_counts = in_window['stop_id'].value_counts().reset_index()

In [45]:
stop_counts.columns = ['stop_id', 'num_departures']

In [46]:
# 7. 停留所名と結合
stop_names = feed.stops[['stop_id', 'stop_name']]
result = pd.merge(stop_counts, stop_names, on='stop_id', how='left')

In [47]:
result

,stop_id,num_departures,stop_name
